In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.preprocessing import image
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import MaxPool2D,Conv2D,Dense,Dropout,Flatten

from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
train_df = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
train_df.label.value_counts()

In [ ]:
plt.figure(figsize=(15,10))
plt.imshow(Image.open('../input/sign-language-mnist/amer_sign2.png'))
plt.axis('off');

In [ ]:
alp = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

pred_class = []
for i in alp:
    pred_class.append(i)

pred_class

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=train_df,x='label');

In [ ]:
y_train = train_df['label']
x_train = train_df.drop(columns=['label'])

y_test = test_df['label']
x_test = test_df.drop(columns=['label'])

print(x_train.shape,y_train.shape)

In [ ]:
x_train.head()

In [ ]:
x_train.shape[0]

In [ ]:
x_train = x_train.values.reshape(x_train.shape[0],28,28,1)
x_test = x_test.values.reshape(x_test.shape[0],28,28,1)

print(x_train.shape,x_test.shape)

**Image Data Generator**

In [ ]:
traingen = Imgen(rescale = 1./255,
                zoom_range=0.2,
                 shear_range=0.2,
                 brightness_range=[0.85,1.20],
                 rotation_range=20,
                 validation_split=0.2
                )
testgen = Imgen(rescale = 1./255)

In [ ]:
train_ds = traingen.flow(x_train,y_train,
                        batch_size = 32,
                        subset='training'
                        )

val_ds = traingen.flow(x_train,y_train,
                        batch_size = 32,
                        subset='validation'
                        )

test_ds = testgen.flow(x_test,y_test,
                       batch_size = 32,
                       shuffle = False
                      )

In [ ]:
x,y = next(train_ds)

In [ ]:
#plot function
def plot_images(images_arr,labels):
    plt.figure(figsize=(15,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(images_arr[i])
        plt.title(pred_class[(labels[i])])
        plt.axis('off')

In [ ]:
plot_images(x,y)

**Model**

In [ ]:
model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    MaxPool2D(2,2),
    
    Conv2D(32,(3,3),activation='relu'),
    MaxPool2D(2,2),
    
    Conv2D(128,(3,3),activation='relu'),
    MaxPool2D(2,2),
    
    Flatten(),
    
    Dense(128,activation='relu'),
    Dropout(0.5),
    
    Dense(64,activation='relu'),
    Dropout(0.5),
    
    Dense(25,activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
#compile
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
keras.utils.plot_model(model,
                      show_shapes=True,
                      show_dtype=True,
                      show_layer_names=True)

In [ ]:
#callbacks
my_calls = [keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3),
            keras.callbacks.ModelCheckpoint("Model.h5",verbose=1,save_best_only=True)]

**Train**

In [ ]:
hist = model.fit(train_ds, epochs=17, validation_data=val_ds, callbacks=my_calls)

In [ ]:
#test
model.evaluate(test_ds,verbose=1)

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(hist.epoch,hist.history['accuracy'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.epoch,hist.history['loss'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()

**Testing**

In [ ]:
preds = model.predict(test_ds)

In [ ]:
preds = [np.argmax(i) for i in preds]

In [ ]:
print(classification_report(preds,y_test))

In [ ]:
#confusion matrix
plt.figure(figsize=(20,15))
sns.heatmap(confusion_matrix(preds,y_test),fmt='d',annot=True,cmap='Blues');

**Visualising Results**

In [ ]:
X,Y = next(test_ds)

In [ ]:
#plot function
def test_images(images_arr,labels):
    plt.figure(figsize=(15,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(images_arr[i])
        plt.xlabel("Actual: {}".format(pred_class[(labels[i])]))
        plt.ylabel("Predicted: {}".format(pred_class[preds[i]]))


In [ ]:
test_images(X,Y)